In [1]:
import torch, time
import torch.nn as nn, torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.datasets import STL10
import torchvision.transforms as T
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

# --- Ayarlar ---
cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)
size = 64              # 64×64 çözünürlük
batch_size = 16
demo_samples = 200
epochs = 5

# --- Dataset: İlk 200 Örnek, 64×64 ---
base_ds = STL10(root='./data', split='train', download=True)
subset = Subset(base_ds, list(range(demo_samples)))
tf_in = T.Compose([T.Resize((size,size)), T.Grayscale(num_output_channels=1), T.ToTensor()])
tf_tgt= T.Compose([T.Resize((size,size)), T.ToTensor()])

class DemoDS(Dataset):
    def __init__(self, base, tf_i, tf_t):
        self.base, self.tf_i, self.tf_t = base, tf_i, tf_t
    def __len__(self): return len(self.base)
    def __getitem__(self, i):
        img,_ = self.base[i]
        return self.tf_i(img), self.tf_t(img)

train_ds = DemoDS(subset, tf_in, tf_tgt)
loader   = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# --- Model: Daha da Hafif LiteColorizer ---
class TinyColorizer(nn.Module):
    def __init__(self):
        super().__init__()
        enc, ch = [], 1
        for oc in [16,32,64,128]:
            enc += [nn.Conv2d(ch,oc,3,padding=1), nn.ReLU(True), nn.BatchNorm2d(oc), nn.MaxPool2d(2)]
            ch = oc
        self.encoder = nn.Sequential(*enc)
        dec = []
        for oc in [64,32,16]:
            dec += [nn.ConvTranspose2d(ch,oc,3,2,1,1), nn.ReLU(True), nn.BatchNorm2d(oc)]
            ch = oc
        dec += [nn.ConvTranspose2d(ch,16,3,2,1,1), nn.ReLU(True), nn.BatchNorm2d(16),
                nn.Conv2d(16,3,3,padding=1), nn.Sigmoid()]
        self.decoder = nn.Sequential(*dec)
    def forward(self,x): return self.decoder(self.encoder(x))

model = TinyColorizer().to(device)
print(model)

# --- Eğitim Döngüsü (5 Epoch Demo) ---
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = GradScaler()

for ep in range(1, epochs+1):
    start = time.time()
    model.train()
    running = 0.0
    for inp, tgt in loader:
        inp, tgt = inp.to(device), tgt.to(device)
        optimizer.zero_grad()
        with autocast():
            out = model(inp)
            loss = criterion(out, tgt)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running += loss.item() * inp.size(0)
    avg_loss = running / len(loader.dataset)
    torch.cuda.synchronize()
    print(f"Epoch {ep}/{epochs} | Loss: {avg_loss:.4f} | Time: {time.time()-start:.1f}s")

print("Demo training complete.")


Device: cuda
TinyColorizer(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Ba

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2072\576141479.py:60: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2072\576141479.py:69: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/5 | Loss: 0.0811 | Time: 0.9s
Epoch 2/5 | Loss: 0.0708 | Time: 0.2s
Epoch 3/5 | Loss: 0.0640 | Time: 0.2s
Epoch 4/5 | Loss: 0.0589 | Time: 0.2s
Epoch 5/5 | Loss: 0.0539 | Time: 0.2s
Demo training complete.


In [3]:
import numpy as np
import torchvision.transforms.functional as F
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
%pip install -q scikit-image

model.eval()
mse_list, psnr_list, ssim_list = [], [], []

for inp, tgt in test_loader:  # test_loader: 64×64 ve batch=16
    inp, tgt = inp.to(device), tgt.to(device)
    with torch.no_grad():
        out = model(inp)
    out_np = out.cpu().numpy()
    tgt_np = tgt.cpu().numpy()
    for i in range(out_np.shape[0]):
        mse_val = ((out_np[i] - tgt_np[i])**2).mean()
        psnr_val = psnr(tgt_np[i].transpose(1,2,0), out_np[i].transpose(1,2,0), data_range=1)
        ssim_val = ssim(tgt_np[i].transpose(1,2,0), out_np[i].transpose(1,2,0), multichannel=True, data_range=1)
        mse_list.append(mse_val)
        psnr_list.append(psnr_val)
        ssim_list.append(ssim_val)

print("Test MSE:", np.mean(mse_list))
print("Test PSNR:", np.mean(psnr_list))
print("Test SSIM:", np.mean(ssim_list))


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


NameError: name 'test_loader' is not defined